In [205]:
import numpy as np
import pandas as pd
from ast import literal_eval
import json
from scipy.spatial.distance import pdist
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

In [254]:
df = pd.read_csv('./data/model_fitting_outputs/summary_data_fit_true.csv')


def calc_accuracy(ground_truth_str, MAP_str):
    ground_truth = np.array(" ".join(ground_truth_str[1:-1].split(' ')).split(), dtype=float)
    MAP = np.array(" ".join(MAP_str[1:-1].split(' ')).split(), dtype=float)

    acc = 1-pdist(np.stack((ground_truth, MAP)))[0] / np.linalg.norm(abs(np.array(ground_truth)) + 2*np.ones((1, 6)))

    return acc

df['accuracy'] = df.apply(lambda x: calc_accuracy(x.ground_truth, x.posterior_map), axis=1)
print(df.model_name.unique())
print(df[df.model_name=='normative_&_1'].accuracy.mean())

['LC_discrete_attention_&_att' 'LC_discrete_attention_&_att_prior'
 'change_d_obs_fk_&_att_cha' 'change_d_obs_fk_&_att_cha_prior'
 'change_d_obs_cause_effect_&_att_cha' 'change_d_obs_cause_&_att_cha'
 'LC_discrete_&_1' 'LC_discrete_&_prior' 'normative_&_1'
 'normative_&_prior' 'ces_strength_&_1' 'ces_no_strength_&_1']
0.9738071242961893


In [249]:
df = pd.read_csv('./data/model_fitting_outputs/summary_data_1.csv')


def calc_accuracy(ground_truth_str, MAP_str):
    ground_truth = np.array(" ".join(ground_truth_str[1:-1].split(' ')).split(), dtype=float)
    MAP = np.array(" ".join(MAP_str[1:-1].split(' ')).split(), dtype=float)

    acc = 1-pdist(np.stack((ground_truth, MAP)))[0] / np.linalg.norm(abs(np.array(ground_truth)) + 2*np.ones((1, 6)))

    return acc

def vec_to_mat(vec):
    K = 3
    idx = 0
    mat = np.zeros((K, K))
    for i in range(K):
        for j in range(K):
            if i == j:
                continue
            
            mat[i, j] = vec[idx]
            idx += 1

    return mat

def mat_to_vec(mat):
    vec = np.zeros(mat.shape[0]**2 - mat.shape[0])
    
    idx = 0
    for i in range(K):
        for j in range(K):
            if i == j:
                continue
            
            vec[idx] = mat[i, j]
            idx += 1

    return vec

def str_to_arr(model_str):
    return np.array(" ".join(model_str[1:-1].split(' ')).split(), dtype=float)
    
def roll_prior_as_str(prior_judgement, condition):
    if type(prior_judgement) != str:
        return np.nan
    
    if condition != 'congruent':  
        mat = vec_to_mat(str_to_arr(prior_judgement))
        roll_mat = np.roll(mat, (1, 1), axis=(0, 1))
        return mat_to_vec(roll_mat)
    else:
        return mat_to_vec(vec_to_mat(str_to_arr(prior_judgement)))

def roll_prior(prior_judgement, condition):
    if condition != 'congruent':  
        mat = vec_to_mat(prior_judgement)
        roll_mat = np.roll(mat, (1, 1), axis=(0, 1))
        return mat_to_vec(roll_mat)
    else:
        return prior_judgement


df['prior_rolled'] = df.apply(lambda x: roll_prior_as_str(x.prior_judgement, x.difficulty), axis=1)

df['accuracy'] = df.apply(lambda x: calc_accuracy(x.ground_truth, x.posterior_map), axis=1)
print(df.model_name.unique())
print(df[df.model_name=='normative_&_1'].accuracy.mean())

['LC_discrete_attention_&_att' 'LC_discrete_attention_&_att_prior'
 'change_d_obs_fk_&_att_cha' 'change_d_obs_fk_&_att_cha_prior'
 'change_d_obs_cause_effect_&_att_cha' 'change_d_obs_cause_&_att_cha'
 'LC_discrete_&_1' 'LC_discrete_&_prior' 'normative_&_1'
 'normative_&_prior' 'ces_strength_&_1' 'ces_no_strength_&_1']
0.9522752755592641


In [223]:
## Import behavioural experiment
with open('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/global_modelling_data.obj', 'rb') as inFile:
    modelling_data = pickle.load(inFile)


for part, data in modelling_data.items():
    for trial, content in data['trials'].items():
        if trial in ['congruent', 'incongruent', 'implausible']:
            #print(trial, modelling_data[part]['trials'][trial]['prior'], roll_prior(content['prior'], trial))
            modelling_data[part]['trials'][trial]['ground_truth'] = roll_prior(content['prior'], trial)
    
with open('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/global_modelling_data_rolled.obj', 'wb') as outFile:
    pickle.dump(modelling_data, outFile)

In [252]:
df_2 = df[df.experiment == 'experiment_3']

df_2[df_2.difficulty == 'incongruent']

,utid,pid,experiment,difficulty,scenario,model_name,ground_truth,posterior_map,posterior_judgement,prior_judgement,prior_entropy,posterior_entropy_unsmoothed,posterior_entropy,model_specs,prior_rolled,accuracy
8736,3_56cb8858edf8da000b6df354_finance_incongruent,56cb8858edf8da000b6df354,experiment_3,incongruent,finance,LC_discrete_attention_&_att,[ 0 -1 0 0 1 0],[ 0. -1. 0. 0. 1. 0.],[ 0. -1. 0. 0. 0.5 0. ],[ 0. 1. 0. 0. -1. 0.],13.931569,4.987350e-08,4.761362,"{'smoothing': 3.33221225, 'decay_rate': 0.8886...","[-1.0, 0.0, 1.0, 0.0, 0.0, 0.0]",1.000000
8737,3_56cb8858edf8da000b6df354_finance_incongruent,56cb8858edf8da000b6df354,experiment_3,incongruent,finance,LC_discrete_attention_&_att_prior,[ 0 -1 0 0 1 0],[ 0. -1. 0. 0. 1. 0.],[ 0. -1. 0. 0. 0.5 0. ],[ 0. 1. 0. 0. -1. 0.],13.931569,1.719431e-08,4.761356,"{'smoothing': 3.33221436, 'decay_rate': 0.8573...","[-1.0, 0.0, 1.0, 0.0, 0.0, 0.0]",1.000000
8738,3_56cb8858edf8da000b6df354_finance_incongruent,56cb8858edf8da000b6df354,experiment_3,incongruent,finance,change_d_obs_fk_&_att_cha,[ 0 -1 0 0 1 0],[ 0. -1. 0.5 0. 1. 0. ],[ 0. -1. 0. 0. 0.5 0. ],[ 0. 1. 0. 0. -1. 0.],13.931567,1.896031e-01,9.179119,"{'smoothing': 2.17430266, 'decay_rate': 0.7471...","[-1.0, 0.0, 1.0, 0.0, 0.0, 0.0]",0.914251
8739,3_56cb8858edf8da000b6df354_finance_incongruent,56cb8858edf8da000b6df354,experiment_3,incongruent,finance,change_d_obs_fk_&_att_cha_prior,[ 0 -1 0 0 1 0],[ 0. -0.5 0. 0. 1. 0. ],[ 0. -1. 0. 0. 0.5 0. ],[ 0. 1. 0. 0. -1. 0.],13.931567,1.115183e-01,9.219157,"{'smoothing': 2.15668694, 'decay_rate': 0.9950...","[-1.0, 0.0, 1.0, 0.0, 0.0, 0.0]",0.914251
8740,3_56cb8858edf8da000b6df354_finance_incongruent,56cb8858edf8da000b6df354,experiment_3,incongruent,finance,change_d_obs_cause_effect_&_att_cha,[ 0 -1 0 0 1 0],[ 0. -1. 0.5 0.5 1. 0.5],[ 0. -1. 0. 0. 0.5 0. ],[ 0. 1. 0. 0. -1. 0.],13.931569,2.011119e+00,10.782301,"{'decay_rate': 0.55776955, 'smoothing': 1.9552...","[-1.0, 0.0, 1.0, 0.0, 0.0, 0.0]",0.851478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14443,3_6176966806de000024ed0ddf_finance_incongruent,6176966806de000024ed0ddf,experiment_3,incongruent,finance,LC_discrete_&_prior,[ 0. -0.5 0.5 -1. 1. 0. ],[ 0. -0.5 0.5 -0.5 0.5 0. ],[ 0.5 -1. -0.5 -0.5 1. 1. ],[ 0.5 1. 0. 0. -0.5 -1. ],13.931569,9.238758e-06,13.661338,"{'smoothing': 0.56517208, 'prior_param': 0.0}","[-0.5, -1.0, 1.0, 0.5, 0.0, 0.0]",0.886039
14444,3_6176966806de000024ed0ddf_finance_incongruent,6176966806de000024ed0ddf,experiment_3,incongruent,finance,normative_&_1,[ 0. -0.5 0.5 -1. 1. 0. ],[ 0. -0.5 0.5 -0.5 0.5 0. ],[ 0.5 -1. -0.5 -0.5 1. 1. ],[ 0.5 1. 0. 0. -0.5 -1. ],13.931569,7.761236e-33,13.931568,{'smoothing': 0.12926732},"[-0.5, -1.0, 1.0, 0.5, 0.0, 0.0]",0.886039
14445,3_6176966806de000024ed0ddf_finance_incongruent,6176966806de000024ed0ddf,experiment_3,incongruent,finance,normative_&_prior,[ 0. -0.5 0.5 -1. 1. 0. ],[ 0. -0.5 0.5 -0.5 0.5 0. ],[ 0.5 -1. -0.5 -0.5 1. 1. ],[ 0.5 1. 0. 0. -0.5 -1. ],13.931569,7.761236e-33,13.931568,"{'smoothing': 0.12873377, 'prior_param': 0.0}","[-0.5, -1.0, 1.0, 0.5, 0.0, 0.0]",0.886039
14446,3_6176966806de000024ed0ddf_finance_incongruent,6176966806de000024ed0ddf,experiment_3,incongruent,finance,ces_strength_&_1,[ 0. -0.5 0.5 -1. 1. 0. ],[-1. -1. -1. -1. -1. -1.],[ 0.5 -1. -0.5 -0.5 1. 1. ],[ 0.5 1. 0. 0. -0.5 -1. ],NaN,NaN,NaN,"{'ce_threshold': 0.61999501, 'time_threshold':...","[-0.5, -1.0, 1.0, 0.5, 0.0, 0.0]",0.530129


In [255]:
# Generate long form dataframes for normative and lc

cols = ['utid', 'pid', 'experiment', 'difficulty', 'scenario', 'accuracy']



# Normative
normative_df = df[df.model_name == 'normative_&_1'][cols].rename({'pid': 'participant'}, axis=1)
normative_df = normative_df[normative_df.scenario.isin(['crime', 'estate', 'finance'])]
normative_df[normative_df.experiment == 'experiment_2'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/normative_lf_2.csv')
normative_df[normative_df.experiment == 'experiment_3'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/normative_lf_3.csv')

# Normative prior
normative_df = df[df.model_name == 'normative_&_prior'][cols].rename({'pid': 'participant'}, axis=1)
normative_df = normative_df[normative_df.scenario.isin(['crime', 'estate', 'finance'])]
normative_df[normative_df.experiment == 'experiment_2'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/normative_p__lf_2.csv')
normative_df[normative_df.experiment == 'experiment_3'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/normative_p_lf_3.csv')

# LC
lc_df = df[df.model_name == 'LC_discrete_&_1'][cols].rename({'pid': 'participant'}, axis=1)
lc_df = lc_df[lc_df.scenario.isin(['crime', 'estate', 'finance'])]
lc_df[lc_df.experiment == 'experiment_2'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/lc_lf_2.csv')
lc_df[lc_df.experiment == 'experiment_3'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/lc_lf_3.csv')

# LC att
lc_att_df = df[df.model_name == 'LC_discrete_attention_&_att'][cols].rename({'pid': 'participant'}, axis=1)
lc_att_df = lc_att_df[lc_att_df.scenario.isin(['crime', 'estate', 'finance'])]
lc_att_df[lc_att_df.experiment == 'experiment_2'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/lc_att_lf_2.csv')
lc_att_df[lc_att_df.experiment == 'experiment_3'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/lc_att_lf_3.csv')

# Change
change_df = df[df.model_name == 'change_d_obs_fk_&_att_cha'][cols].rename({'pid': 'participant'}, axis=1)
change_df = change_df[change_df.scenario.isin(['crime', 'estate', 'finance'])]
change_df[change_df.experiment == 'experiment_2'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/change_lf_2.csv')
change_df[change_df.experiment == 'experiment_3'].to_csv('/mnt/c/Users/vbtes/CompProjects/vbtCogSci/csl_global_analysis/data/simulated_data/change_lf_3.csv')


In [251]:
lc_df

,utid,participant,experiment,difficulty,scenario,accuracy
30,1_56da8da8c5b248000ae2adaf_crime_label,56da8da8c5b248000ae2adaf,experiment_1,label,crime,1.000000
126,1_5999ad2a0ed7ae0001991b57_crime_label,5999ad2a0ed7ae0001991b57,experiment_1,label,crime,1.000000
222,1_5b8bbd437073290001839123_crime_label,5b8bbd437073290001839123,experiment_1,label,crime,0.919418
270,1_5bcdd6be25295a0001a71beb_crime_label,5bcdd6be25295a0001a71beb,experiment_1,label,crime,1.000000
318,1_5be87e33a14b540001aa2ea5_crime_label,5be87e33a14b540001aa2ea5,experiment_1,label,crime,1.000000
...,...,...,...,...,...,...
14346,3_616c3f3a378cb38f3f7ecac3_finance_congruent,616c3f3a378cb38f3f7ecac3,experiment_3,congruent,finance,1.000000
14370,3_61717173748006894b2b54ff_crime_incongruent,61717173748006894b2b54ff,experiment_3,incongruent,crime,0.856408
14394,3_61717173748006894b2b54ff_finance_congruent,61717173748006894b2b54ff,experiment_3,congruent,finance,0.867932
14418,3_6176966806de000024ed0ddf_crime_congruent,6176966806de000024ed0ddf,experiment_3,congruent,crime,0.737387


In [195]:
# Generate actions

N = 300

abs_value = 90
time = 1/4
K = 3

def generate_action_plan(N, K=3, abs_value=90, time=2/7):
    split = time*np.ones((2, K))

    split[1, :] = ( 1 - K*time) / K

    split_units = (N * split).astype(int)
    split_units
    done_counts = np.zeros(split_units.shape)

    actions = [None for _ in range(N)]
    values = np.zeros((N, K))

    for i in range(N):
        for j in range(K):
            if done_counts[0, j] < split_units[0, j]:
                actions[i] = j
                values[i, j] = abs_value if done_counts[0, j] < split_units[0, j] / 2 else - abs_value
                done_counts[0, j] += 1
                break
            elif done_counts[1, j] < split_units[1, j]:
                done_counts[1, j] += 1
                break

    return actions, values





In [196]:

a = vec_to_mat(np.array([1, 2, 3, 4, 5, 6]))
mat_to_vec(a)

array([1., 2., 3., 4., 5., 6.])

In [181]:
df_manip = df[df.experiment == 'experiment_3']
df_manip = df_manip[df_manip.difficulty.isin(['congruent', 'incongruent', 'implausible'])]

df_manip[df_manip.model_name == 'normative_&_1']

results = []
for i, idx in enumerate(df_manip.index):
    mat_gt = vec_to_mat(str_to_arr(df_manip.loc[idx, 'ground_truth']))
    mat_p = vec_to_mat(str_to_arr(df_manip.loc[idx, 'prior_judgement']))

    if df_manip.loc[idx, 'difficulty'] == 'congruent':
        results.append((mat_gt == mat_p).sum())

    else:
        results.append((mat_gt == mat_p.T).sum())

        
        
np.array(results).mean()

9.0

In [188]:
a = np.array([1, 0, 0, 1/2, -1/2, 0])

a = a.reshape((3, 2))

print(a)
b = np.zeros(a.shape)
b[:, 0] = a[:, 1]
b[:, 1] = a[:, 0]

print(np.roll(a, (1, 1), axis=(0, 1)))

[[ 1.   0. ]
 [ 0.   0.5]
 [-0.5  0. ]]
[[ 0.  -0.5]
 [ 0.   1. ]
 [ 0.5  0. ]]
